In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [ ]:
from tqdm import tqdm
from pathlib import Path
from src.utils.config_loader import load_config

base_dir = Path(os.getcwd()).parent

config = load_config(base_dir / 'secrets.yaml')

In [3]:
from src.data.preprocessing import create_df

val_df = create_df(base_dir / 'data/my_data/regplans-dev.conllu')

In [4]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import (SystemMessage, HumanMessage, AIMessage)

os.environ['OPENAI_API_VERSION'] = config['OPENAI_API_VERSION']
os.environ['AZURE_OPENAI_ENDPOINT'] = config['OPENAI_API_BASE']
os.environ['AZURE_OPENAI_API_KEY'] = config['OPENAI_API_KEY']

llm = AzureChatOpenAI(
    deployment_name=config['OPENAI_DEPLOYMENT_NAME']
)#.bind(response_format={"type": "json_object"})

In [ ]:
import random
import json

with open(base_dir / 'llm_stuff/prompts/examples.json', 'r') as f:
    example_bank = json.load(f)

def format_examples(example_subset): 
    # Formats the examples into a string for later prompt
    formatted = []
    for i, ex in enumerate(example_subset):
        entity_lines = "\n".join([f"{e['word']} {e['label']}" for e in ex["entities"]])
        formatted.append(f"Example {i+1}:\nSentence: \"{ex['sentence']}\"\nEntities:\n{entity_lines}\n")
    
    return "\n".join(formatted)

the_examples = random.sample(example_bank, 5)
formatted_examples = format_examples(the_examples)

In [ ]:
from deap import base, creator, tools, algorithms

# What information should i keep from each run?

In [ ]:
from src.utils.label_mapping_regplans import label_to_id
from collections import defaultdict
from textwrap import dedent

all_pred_ids = []
all_true_ids = []
all_results = []

for idx, row in tqdm(val_df[:5].iterrows(), total=len(val_df[:5])):

    sentence = row['full_text']
    tokens = row['words']
    true_labels = row['labels']  

    msg = [
    SystemMessage(
        f"""You are an expert in Natural Language Processing. Your task is to identify Named Entities (NER) in a given text.
            The possible Named Entities are exclusively 'B-FELT' and 'I-FELT'. The entities are defined as follows:

            - B-FELT: The beginning of a field zone name.
            - I-FELT: The continuation of a field zone name.    

            Important Rules:
            - A B-FELT must always appear before an I-FELT.
            - An I-FELT cannot exist without a preceding B-FELT.
                                
            Below are some examples of sentences with their corresponding entities:

            {formatted_examples}
        """
    ),
    HumanMessage(f"Your task is to identify the Named Entities in the following sentence: '{sentence}'") ]

    response = llm.invoke(msg)

    entities = defaultdict(list) # Word-label pairs

    for line in response.content.splitlines():
        parts = line.strip().split()
        if len(parts) == 2:
            word, label = parts[0], parts[1]
            entities[word].append(label)

    pred_labels = []
    word_counts = defaultdict(int)  # Track occurrences of each word

    for token in tokens:
        if token in entities and word_counts[token] < len(entities[token]):
            pred_labels.append(entities[token][word_counts[token]])  # Get the label in order
            word_counts[token] += 1  # Increment occurrence counter
        else:
            pred_labels.append("O")  # Default to "O" if missing

    # Convert labels to IDs
    pred_ids = []
    for label in pred_labels:
        if label in label_to_id:
            pred_ids.append(label_to_id[label])
        else:
            print(f"Warning: Unexpected label '{label}' found. Assigning default label 'O'.")
            pred_ids.append(label_to_id.get("O", -1))

    true_ids = [label_to_id[label] for label in true_labels]

    all_pred_ids.extend(pred_ids)
    all_true_ids.extend(true_ids)

    all_results.append({
        'sentence': sentence,
        'tokens': tokens,
        'true_labels': true_labels,
        'predicted_labels': pred_labels,
        'generated_text': response.content
    })

100%|██████████| 5/5 [00:01<00:00,  2.72it/s]


In [ ]:
from llm_stuff.evaluation import evaluate 

metrics = evaluate(all_true_ids, all_pred_ids)

print("Evaluation Metrics on Dev Set:")
print(metrics)

final_output = {
    'prompt': str(msg),
    'evaluation_metrics': metrics,
    'results': all_results
}

with open(base_dir / f'llm_stuff/results/{config['OPENAI_DEPLOYMENT_NAME']}.json', 'w', encoding='utf-8') as f:
    json.dump(final_output, f, indent=4, ensure_ascii=False)

Evaluation Metrics on Dev Set:
{'precision': 0.8597659468650818, 'recall': 0.7329059839248657, 'f1': 0.7816230654716492, 'span_acc': 0.7769230604171753, 'classification_report': {'B-FELT': {'precision': 0.9298245614035088, 'recall': 0.8153846153846154, 'f1-score': 0.8688524590163934, 'support': 130.0}, 'I-FELT': {'precision': 0.6875, 'recall': 0.39285714285714285, 'f1-score': 0.5, 'support': 28.0}, 'O': {'precision': 0.9619732785200411, 'recall': 0.9904761904761905, 'f1-score': 0.9760166840458812, 'support': 945.0}, 'accuracy': 0.9546690843155031, 'macro avg': {'precision': 0.8597659466411832, 'recall': 0.732905982905983, 'f1-score': 0.7816230476874249, 'support': 1103.0}, 'weighted avg': {'precision': 0.9512166284532141, 'recall': 0.9546690843155031, 'f1-score': 0.9513024352633623, 'support': 1103.0}}}
